### Using Kalman Filters

Similar to the previously used Monte-Carlo simulation, but using a continuous environment and thus continuous probability distributions.

Creates a uni-modal disitribution, and uses Gaussian distribution as its probability density functions. The Gaussian only comes into play to provide a mean (expectation value) and a variance to use as the uncertainty.

### Updating a Gaussian distribution multilpicatively by Bayes' rule:

New mean is the weighted average of the prior $(\mu,\sigma^2)$ and likelihood $(\nu,r^2)$ means: $\mu^{'}=\frac{\sigma^2\nu+r^2\mu}{\sigma^2+r^2}$.

The new standard deviation is given by: $\sigma^{'2}=\frac{1}{\frac{1}{\sigma^2}+\frac{1}{r^2}}$.

This is where Bayesian inference comes into the Kalman filter - the Kalman filter carries out the same operations with matrices, without ever explicitly incorporating probabilities or probabilistic inference anywhere.

The Kalman filter takes the brute force, common sense approach, of allowing an update in the estimated values each iteration by a fraction of the difference from the measured value. The size of the fraction ($0\geq K\geq1$) is decided by the previous estimate's and observation's relative uncertainties. 

### What is a Kalman Filter?

It is a method of updating the velocity and position space with a mean point $(E(X_t),E(\dot{X_t}))$ and a variance described by the covariance matrix. 

Every measurement (sensing step) updates the mean and variance for step $t$ by, effectively, Bayesian Inference, using the Gaussian multiplicativity rules above, and the movement step linearly adds the motion to the mean and variance for $t+1$.

The uncertainty in velocity can decrease with measurement of position only due to the dependence of position on velocity (not at all statistically independent as $x^{'}=x+\dot{x}\delta t$). The velocity is a $\textbf{hidden variable}$, and the position an $\textbf{observed variable}$.

##### The likelihood (measurement pdf) is a Gaussian with high variance in the velocity axis and small in the position axis (as it is position that is measured in the sensing step), and the prior function is a Gaussian with greates covariance in the joint velocity, space line (uncertainty both in velocity and space). The product posterior distribution is more centralised in both the velocity and space axes.

## Mathematical implementation:

#### Derived `retrospectively`, i.e., know the desired outcome (a filter), how to build that using matrices - don't overthink it.
#### eg. see iLectureOnline videos, text document and blackboard photos.

Note $H=[1\text{ }0]$ often, and $F=[[1\text{ }\delta t],[0\text{ }1]]$. $C$ is the covariance matrix.

$S=HCH^{T}+R$ represents the sum of the estimation error and the measurement error, and is used to normalise (hence the inverse) the estimation error in the construction of the Kalman Filter (gate).

#### For motion:

$X_{t+1}=FX_{t}+U_{t}$ for $F$ the 'state transition matrix' and $U_{t}$ the 'external motion vector', i.e. the motion of the sensor and not the object being tracked (introduces an idea of acceleration).

$C_{t+1}=FC_{t}F^{T}$.

$F$ incorporates the motion: $X^{t+1}_{1}=F_{11}X^{t}_{1}+F^{t}_{12}X_{2}$, 

i.e. $x=F_{11}x+F_{12}\dot{x},\text{ }F_{12}=\delta t$ and $F_{11}=1$. Note that expected velocity doesn't depend explicitly on position (only second derivative), so the update for $\dot{x}_{t+1}$ is dependent only on the earlier of $\dot{x}_{t}$. 

Note also that with respect to velocity, the state transition matrix represents a martingale as $F_{22}=1$, i.e., the expectation for the next velocity is the previous time step's expectation value.

#### For sensing:

($t+1$ becomes $t$ if move first, sense second).

$K_{t}=C_{t}H_{t}^{T}S_{t}^{-1}$ is the Kalman filter, and $H$ is the measurement function, and $S_{t}=H_{t}C_{t}H_{t}^{T}+R_{t}$, for $R_{t}$ the measurement error.

$Y_{t}=Z_{t}-H_{t}X_{t}$, for $Z$ the measurement.

$X_{t}^{'}=X_{t}+K_{t}Y_{t}$ - The new position. $K$ acts like a gate, dictating how much the new measurement will influence the next iteration's estimate based on how small the measurement uncertainty is relative to the current estimate's uncertainty.

$C_{t}^{'}=(I-K_{t}H_{t})C_{t}$ - The new covariance matrix.

In [98]:
from math import *


class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise ValueError("Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise ValueError("Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError("Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError("Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise ValueError("Matrices must be m*n and n*p to multiply")
        else:
            # multiply if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise ValueError("Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                    res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                    raise ValueError("Zero diagonal")
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        if self.value:
            print(self.value)
            return repr(self.value)

In [99]:
def kalman_filter(x, P):
    # Measure then predict (move) order.
    for n in range(len(measurements)):
        # Sense:
        S = H * P * H.transpose() + R # Dimensions of position (eg. scalar in 1D)
        K = P * H.transpose() * S.inverse() # Dimension of Kalman space (eg. 2x1 vector in 1D)
        Y = matrix([measurements[n]]).transpose() - H * x # Dimensions of position (eg. scalar in 1D)
        x += K * Y # Dimension of Kalman space (eg. 2x1 vector in 1D)
        P = (I - K * H) * P # (approximate) covariance matrix is square with dimensions of Kalman space (eg. 2x2 for 1D)
        
        # Move:
        x = F * x + u # Dimension of Kalman space (eg. 2x1 vector in 1D)
        P = F * P * F.transpose() # (approximate) covariance matrix is square with dimensions of Kalman space (eg. 2x2 for 1D)
    return x,P

In [100]:
## Test: - 1 dimensional motion in 2 dimensionsal kalman space
measurements = [[1],[2],[3]]

x = matrix([[0.], [0.]]) # initial state (location and velocity)
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., 1.], [0, 1.]]) # next state function
H = matrix([[1., 0.]]) # measurement function
R = matrix([[1.]]) # measurement uncertainty
I = matrix([[1., 0.], [0., 1.]]) # identity matrix

print('Kalman state space for 1D motion:')
print(kalman_filter(x, P)[0].show())

print('Covariance matrix:')
print(kalman_filter(x, P)[1].show())

Kalman state space for 1D motion:
[3.9996664447958645]
[0.9999998335552873]
 
None
Covariance matrix:
[2.3318904241194827, 0.9991676099921091]
[0.9991676099921067, 0.49950058263974184]
 
None


Notice that over time (discrete time-steps), the velocity becomes correllated (greatly) with the position due to the large amount of statistical dependence of position on velocity (linked explicitly in the equations of motion). This is even though the initialisation of the covariance matrix had no assumed correllation between position and velocity. 

The system `learns` about the correlation because of the dependence inherited from the state transition matrix, $F$.

#### 2D Euclidean space = 4D Kalman space:

In [101]:
measurements = [[1,-2],[2,-4],[3,-6],[4,-8],[5,-10]]

dt = 0.1
sensor_error = 0.1

x = matrix([[0.],[0.],[0.],[0.]]) # initial state (location and velocity)
P = matrix([[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,1000.,0.], [0.,0.,0.,1000.]]) # initial uncertainty - eg. high in velocities, low in position
u = matrix([[0.],[0.],[0.],[0.]]) # external motion
F = matrix([[1.,0.,dt,0.],[0.,1.,0.,dt],[0.,0.,1.,0.],[0.,0.,0.,1.]]) # next state function
H = matrix([[1.,0.,0.,0.],[0.,1.,0.,0.]]) # measurement function
R = matrix([[sensor_error,0.],[0.,sensor_error]]) # measurement uncertainty - applied to position measurements only, uncertainty in x and y.
I = matrix([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,1.,0.],[0.,0.,0.,1.]]) # identity matrix

print('Kalman state space for 2D motion:')
print(kalman_filter(x, P)[0].show())

print('Covariance matrix:')
print(kalman_filter(x, P)[1].show())

Kalman state space for 2D motion:
[6.6644451849383515]
[-13.328890369876703]
[13.328890369876694]
[-26.657780739753388]
 
None
Covariance matrix:
[0.08330556481172977, 0.0, 0.16661112962346075, 0.0]
[0.0, 0.08330556481172977, 0.0, 0.16661112962346075]
[0.1666111296234606, 0.0, 0.33322225924692717, 0.0]
[0.0, 0.1666111296234606, 0.0, 0.33322225924692717]
 
None
